# UNet++ & Unet for microwell-DL

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
import cv2
from tqdm import tqdm
from time import time
import datetime
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.layers import Input, Add, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import callbacks

from keras import callbacks
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint


from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from IPython.display import clear_output
from IPython.display import HTML
from base64 import b64encode

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set yout own directory
os.chdir('/content/drive/MyDrive/*********/')

In [ ]:
# Load directories
train_data_dir = "./Training dataset/Training_source/"

## Train test set seperation

In [ ]:
# Number of training examples

TRAINSET_SIZE = int(round(len(os.listdir(train_data_dir)) * 0.7))
print(f"Number of Training Examples: {TRAINSET_SIZE}")

VALIDSET_SIZE = int(len(os.listdir(train_data_dir)) * 0.1)
print(f"Number of Validation Examples: {VALIDSET_SIZE}")

TESTSET_SIZE = int(len(os.listdir(train_data_dir)) - TRAINSET_SIZE - VALIDSET_SIZE)
print(f"Number of Testing Examples: {TESTSET_SIZE}")

In [ ]:
# Initialize Constants
IMG_SIZE = 128
N_CHANNELS = 3
N_CLASSES = 1
SEED = 123

In [ ]:

# Function to load image and return a dictionary
def parse_image(img_path: str) -> dict:
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels=3)


    mask_path = tf.strings.regex_replace(img_path,"Training_source", "Training_target")
    mask_path = tf.strings.regex_replace(mask_path, "well", "mask")
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)


    bac_label = np.array([255, 255, 255])


    # Convert to mask to binary mask
    bac_label = np.array([255, 255, 255])
    mask = tf.experimental.numpy.all(mask == bac_label, axis = 2)
    mask = tf.cast(mask, tf.uint8)
    mask = tf.expand_dims(mask, axis=-1)

    return {'image': image, 'segmentation_mask': mask}





# Generate dataset variables

In [ ]:
BATCH_SIZE = 15
BUFFER_SIZE = 1000

### Generate dataset variables
all_dataset = tf.data.Dataset.list_files(train_data_dir + "*.png",  shuffle = False)
all_dataset = all_dataset.shuffle(BUFFER_SIZE, seed=SEED, reshuffle_each_iteration=False)
all_dataset = all_dataset.map(parse_image)


train_dataset = all_dataset.take(TRAINSET_SIZE + VALIDSET_SIZE)
val_dataset = train_dataset.skip(TRAINSET_SIZE)
train_dataset = train_dataset.take(TRAINSET_SIZE)
test_dataset = all_dataset.skip(TRAINSET_SIZE + VALIDSET_SIZE)

In [ ]:
# Tensorflow function to rescale images to [0, 1]
@tf.function
def normalize(input_image: tf.Tensor, input_mask: tf.Tensor) -> tuple:
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image, input_mask

# Tensorflow function to apply preprocessing transformations
@tf.function
def load_image_train(datapoint: dict) -> tuple:
    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.math.round(input_mask)

    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)

    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask

# Tensorflow function to preprocess validation images
@tf.function
def load_image_test(datapoint: dict) -> tuple:
    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.math.round(input_mask)
    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask

In [ ]:
dataset = {"train": train_dataset, "val": val_dataset, "test": test_dataset}

# -- Train Dataset --#
dataset['train'] = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.AUTOTUNE)
dataset['train'] = dataset['train'].shuffle(buffer_size=BUFFER_SIZE, seed=SEED)
dataset['train'] = dataset['train'].repeat()
dataset['train'] = dataset['train'].batch(BATCH_SIZE)
dataset['train'] = dataset['train'].prefetch(buffer_size=tf.data.AUTOTUNE)

#-- Validation Dataset --#
dataset['val'] = dataset['val'].map(load_image_test)
dataset['val'] = dataset['val'].repeat()
dataset['val'] = dataset['val'].batch(BATCH_SIZE)
dataset['val'] = dataset['val'].prefetch(buffer_size=tf.data.AUTOTUNE)

#-- Testing Dataset --#
dataset['test'] = dataset['test'].map(load_image_test)
dataset['test'] = dataset['test'].batch(BATCH_SIZE)
dataset['test'] = dataset['test'].prefetch(buffer_size=tf.data.AUTOTUNE)

print(dataset['train'])
print(dataset['val'])
print(dataset['test'])

In [ ]:
# Function to view the images from the directory
def display_sample(display_list):
    plt.figure(figsize=(18, 18))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')

    plt.show()

In [ ]:
# View the training images from the directory
for image, mask in dataset['train'].take(1):
    sample_image, sample_mask = image, mask

display_sample([sample_image[0], sample_mask[0]])

# Model: UNet++

## Set Unet++ Result folder

In [ ]:
unetPP_folder = 'UNetPP_result/'

## UNet++ Architechture


In [ ]:
def double_conv_block(x, n_filters):
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   return x

def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = layers.MaxPool2D(2)(f)
   p = layers.Dropout(0.5)(p)
   return f, p


def upsample_block_part1(x, n_filters):
   # upsample
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   return x

def upsample_block_part2(x, n_filters):
   # dropout
   x = layers.Dropout(0.5)(x)
   # Conv2D twice with ReLU activation
   x = double_conv_block(x, n_filters)
   return x


def build_unetPP_model():
    # inputs
    inputs = layers.Input(shape=(128,128,3))

    # encoder: contracting path - downsample
    f00, p00 = downsample_block(inputs, 64)
    f10, p10 = downsample_block(p00, 128)
    f20, p20 = downsample_block(p10, 256)
    f30, p30 = downsample_block(p20, 512)
    bottleneck = double_conv_block(p30, 1024)

    # decoders: expanding path - upsample
    u01 = upsample_block_part1 (p00, 64)
    u01 = layers.concatenate ([u01, f00])
    u01 = upsample_block_part2 (u01, 64)

    u11 = upsample_block_part1 (p10, 128)
    u11 = layers.concatenate ([u11, f10])
    u11 = upsample_block_part2 (u11, 128)

    u21 = upsample_block_part1 (p20, 256)
    u21 = layers.concatenate ([u21, f20])
    u21 = upsample_block_part2 (u21, 256)

    u31 = upsample_block_part1 (bottleneck, 512)
    u31 = layers.concatenate ([u31, f30])
    u31 = upsample_block_part2 (u31, 512)

    u02 = upsample_block_part1 (u11, 64)
    u02 = layers.concatenate ([u02, f00, u01])
    u02 = upsample_block_part2 (u02, 64)

    u12 = upsample_block_part1 (u21, 128)
    u12 = layers.concatenate ([u12, f10, u11])
    u12 = upsample_block_part2 (u12, 128)

    u22 = upsample_block_part1 (u31, 256)
    u22 = layers.concatenate([u22, f20, u21])
    u22 = upsample_block_part2(u22, 256)

    u03 = upsample_block_part1 (u12, 64)
    u03 = layers.concatenate([u03, f00, u01, u02])
    u03 = upsample_block_part2(u03, 64)

    u13 = upsample_block_part1 (u22, 128)
    u13 = layers.concatenate([u13, f10, u11, u12])
    u13 = upsample_block_part2(u13, 128)

    u04 = upsample_block_part1 (u13, 64)
    u04 = layers.concatenate([u04, f00, u01, u02, u03])
    u04 = upsample_block_part2(u04, 64)

    # outputs
    outputs = layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u04)
    # unet model with Keras Functional API
    unetPP_model = tf.keras.Model(inputs, outputs, name="UNet++")
    return unetPP_model





In [ ]:
model = build_unetPP_model()
model.summary()

## Model configuration

In [ ]:
# Optimization
opt = keras.optimizers.Adam(learning_rate=1e-4, clipvalue=0.5)

m_iou = tf.keras.metrics.BinaryIoU(
    target_class_ids=[1], threshold=0.5, name=None, dtype=None
)

model.compile(loss=BinaryCrossentropy(), optimizer=opt,  metrics=[m_iou])

## Training
The segmentation model is trained with fixed 15 epoches. Each epoch containts 100 batches and each batch contains 32 samples.

The training process here is far from systematic, and is provided for illustration purposes only.

In [ ]:
# Callbacks and Logs
class DisplayCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks = [
    DisplayCallback(),
    callbacks.TensorBoard(logdir, histogram_freq = -1),
    callbacks.EarlyStopping(patience = 10, verbose = 1),
    callbacks.ModelCheckpoint(unetPP_folder + 'best_UNetPP_model_0513.ckpt',save_weights_only=True, verbose = 1, save_best_only = True)
]

## Set Epochs **IMPORTANT**
EPOCHS = 300

## Set Variables
STEPS_PER_EPOCH = TRAINSET_SIZE // BATCH_SIZE
VALIDATION_STEPS = VALIDSET_SIZE // BATCH_SIZE
print(TRAINSET_SIZE)
print(VALIDSET_SIZE)
print(BATCH_SIZE)
print(STEPS_PER_EPOCH)
print(VALIDATION_STEPS)



class CustomHistory(Callback):
    def __init__(self):
        self.losses = []
        self.mean_io_u_1_values = []
        self.val_losses = []
        self.val_mean_io_u_1_values = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs.get('loss'))
        self.mean_io_u_1_values.append(logs.get('binary_io_u'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_mean_io_u_1_values.append(logs.get('val_binary_io_u'))


custom_history = CustomHistory()



checkpoint = ModelCheckpoint(unetPP_folder + 'best_UNetPP_model_0513.ckpt', save_weights_only=True, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [custom_history, checkpoint]

In [ ]:
initial_time = time()


# Model Training
model.fit(dataset['train'], epochs=EPOCHS,
          steps_per_epoch=STEPS_PER_EPOCH,
          validation_data=dataset['val'],
          validation_steps=VALIDATION_STEPS,
          callbacks=callbacks_list)


print('Finished Unet++ Training')
print('Training time', time() - initial_time)
model.save_weights(unetPP_folder + 'epoch300_model/Epoch300_UNETPP_model.ckpt')

## validation set  show

In [ ]:
#### Repeat for safety and reabiility #####################################################
# Function to view the images from the directory
def display_sample(display_list):
    plt.figure(figsize=(18, 18))

    title = ['Input Image', 'True Mask', 'Predict Map', 'Predict Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')

    plt.show()

# Function to create a mask out of network prediction
def create_mask(pred_mask: tf.Tensor) -> tf.Tensor:
    # Round to closest
    pred_mask = tf.math.round(pred_mask)

    # [IMG_SIZE, IMG_SIZE] -> [IMG_SIZE, IMG_SIZE, 1]
    pred_mask = tf.expand_dims(pred_mask, axis=-1)
    return pred_mask



# Function to show predictions
def show_validation(dataset=None, num=1):
    if dataset:
        # Predict and show image from input dataset
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display_sample([image[0], true_mask, pred_mask, create_mask(pred_mask)])

            img = tf.keras.preprocessing.image.array_to_img(image[0])
            img.save(unetPP_folder + f'outputs/validate_{num}_well.png')

            true_mask_img = tf.keras.preprocessing.image.array_to_img(true_mask)
            true_mask_img.save(unetPP_folder + f'outputs/validate_{num}_true_mask.png')

            pred_map_img = tf.keras.preprocessing.image.array_to_img(pred_mask)
            pred_map_img.save(unetPP_folder + f'outputs/validate_{num}_predict_map.png')

            pred_mask_img = tf.keras.preprocessing.image.array_to_img(create_mask(pred_mask))
            pred_mask_img.save(unetPP_folder + f'outputs/validate_{num}_predict_mask.png')


    else:
        # Predict and show the sample image
        inference = model.predict(sample_image)
        display_sample([sample_image[0], sample_mask[0],
                        inference[0]])

        img = tf.keras.preprocessing.image.array_to_img(sample_image[0])
        img.save(unetPP_folder + f'outputs/validate_{num}_well.png')

        true_mask_img = tf.keras.preprocessing.image.array_to_img(sample_mask[0])
        true_mask_img.save(unetPP_folder + f'outputs/validate_{num}_true_mask.png')

        pred_mask_img = tf.keras.preprocessing.image.array_to_img(inference[0])
        pred_mask_img.save(unetPP_folder + f'outputs/validate_{num}_predict_mask.png')


## Show validation image, true mask & predict mask
show_validation(dataset['test'], 20)

## From custom_history collect Train_loss & Val_loss

In [ ]:
# Training Loss

epochs = list(range(1, len(training_loss) + 1))

df = pd.DataFrame({"Epochs": epochs, "train_loss" : training_loss})
df.to_csv(unetPP_folder + 'training_loss.csv', index=False)


training_loss = custom_history.losses
print(training_loss)

# generate epochs list
epochs = list(range(1, len(training_loss) + 1))

# Plot Line Chart
plt.figure(figsize=(10, 6))
plt.plot(epochs,training_loss, '-', label='training_loss')
plt.title('training_loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('training_loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Validate Loss

val_losses = custom_history.val_losses


# generate epochs list
epochs = list(range(1, len(val_losses) + 1))

# save log file
df = pd.DataFrame({"Epochs": epochs, "validate_loss" : val_losses})
df.to_csv(unetPP_folder + 'validate_loss.csv', index=False)


# Plot Line Chart
plt.figure(figsize=(10, 6))
plt.plot(epochs,val_losses, '-', label='val_losses')
plt.title('val_losses over epochs')
plt.xlabel('Epochs')
plt.ylabel('val_losses')
plt.legend()
plt.tight_layout()
plt.show()

## 從 custom_history 獲取 train_BIOU & val_BIOU 數據

In [ ]:
# 從 custom_history 獲取 mean_io_u_1_values 數據
mean_io_u_1_values = custom_history.mean_io_u_1_values
print(mean_io_u_1_values)

# 創建 epochs 列表
epochs = list(range(1, len(mean_io_u_1_values) + 1))


df = pd.DataFrame({"Epochs": epochs, "train_bIOU" : mean_io_u_1_values})
df.to_csv(unetPP_folder + 'training_bIOU.csv', index=False)


# 繪製折線圖
plt.figure(figsize=(10, 6))
plt.plot(epochs, mean_io_u_1_values, '-', label='mean_io_u_values')
plt.title('mean_io_u over epochs')
plt.xlabel('Epochs')
plt.ylabel('mean_io_u')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 從 custom_history 獲取 mean_io_u_1_values 數據
val_mean_io_u_1_values = custom_history.val_mean_io_u_1_values
print(val_mean_io_u_1_values)

# 創建 epochs 列表
epochs = list(range(1, len(val_mean_io_u_1_values) + 1))


df = pd.DataFrame({"Epochs": epochs, "val_bIOU" : val_mean_io_u_1_values})
df.to_csv(unetPP_folder + 'validate_bIOU.csv', index=False)


# 繪製折線圖
plt.figure(figsize=(10, 6))
plt.plot(epochs, val_mean_io_u_1_values, '-', label='val_mean_io_u_values')
plt.title('val_mean_io_u_values over epochs')
plt.xlabel('Epochs')
plt.ylabel('val_mean_io_u_values')
plt.legend()
plt.tight_layout()
plt.show()



## 4to1 plot

(0.12, 0.8)

In [ ]:
unetPP_folder = 'UNetPP_result/'

plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 300


df1 = pd.read_csv(unetPP_folder +'training_loss.csv', header=0)
df2 = pd.read_csv(unetPP_folder +'validate_loss.csv', header=0)
df3 = pd.read_csv(unetPP_folder + 'training_bIOU.csv', header=0)
df4 = pd.read_csv(unetPP_folder + 'validate_bIOU.csv', header=0)


fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('Epochs', fontsize=12)
ax1.set_ylabel('Loss', color=color, fontsize=12)
ax1.plot(df1["Epochs"], df1["train_loss"], '-', label='Training Loss', color=color)
ax1.plot(df2["Epochs"], df2["validate_loss"], '-', alpha = 0.5, label='Validation Loss', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_ylim([-0.01, 0.13])
ax1.set_box_aspect(0.75)



ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('IoU', color=color, fontsize=12)  # we already handled the x-label with ax1
ax2.plot(df3["Epochs"], df3["train_bIOU"], '-', label='Training IoU', color=color)
ax2.plot(df4["Epochs"], df4["val_bIOU"], '-', alpha = 0.5, label='Validation IoU', color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax2.set_ylim([-0.1, 0.9])
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=9)


plt.show()

fig.savefig(unetPP_folder + 'Unet++_4to1.png')


# UNET++ Testing

## (Optional) Load pretrained Swin-Unet model weights

In [ ]:
model.load_weights(unetPP_folder + 'epoch300_model/Epoch300_UNETPP_model.ckpt')
model.summary()

## Calculate Time of test-set prediction

In [ ]:
initial_time = time()

for image, mask in dataset['test']:
    pred_mask = model.predict(image)

print('Finished Unet++ Testing')
print('Testing time', time() - initial_time)

## Function to calculate mask over image

In [ ]:
# Function to calculate mask over image
def weighted_img(img, initial_img, α=1., β=0.5, γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

# Function to process an individual image and it's mask
def process_image_mask(image, mask):
    # Round to closest
    mask = tf.math.round(mask)

    # Convert to mask image
    zero_image = np.zeros_like(mask)
    mask = np.dstack((mask, zero_image, zero_image))
    mask = np.asarray(mask, np.float32)

    # Convert to image image
    image = np.asarray(image, np.float32)

    # Get the final image
    final_image = weighted_img(mask, image)

    return final_image

# Function to calculate bacteria number inside microwell

In [ ]:
def bacteria_count(mask):
    mask_img = tf.keras.preprocessing.image.array_to_img(mask)
    img2 = cv2.cvtColor(np.asarray(mask_img), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    bacNum = len(cnts)
    return bacNum

# Function to save predictions

In [ ]:
# Function to save predictions
def save_predictions(dataset, True_Counts, Pred_Counts):
    # Predict and save image the from input dataset
    index = 0
    for batch_image, batch_mask in dataset:
        for image, mask in zip(batch_image, batch_mask):
            print(f"Processing image : {index}")
            pred_mask = model.predict(tf.expand_dims(image, axis = 0))
            save_sample([image, process_image_mask(image, mask), process_image_mask(image, pred_mask[0])], index)

            true_bacNum = bacteria_count(mask)
            True_Counts.append(true_bacNum)
            print("True Bac-Num:    ", true_bacNum)
            print(" ")

            pred_mask_round = tf.math.round(pred_mask[0])
            pred_bacNum = bacteria_count(pred_mask_round)
            Pred_Counts.append(pred_bacNum)
            print("Predict Bac-Num: ", pred_bacNum)
            print(" ")

            img = tf.keras.preprocessing.image.array_to_img(image)
            img.save(unetPP_folder + f'outputs/{index}_well.png')
            true_mask_img = tf.keras.preprocessing.image.array_to_img(process_image_mask(image, mask))
            true_mask_img.save(unetPP_folder + f'outputs/{index}_true.png')
            pred_mask_img = tf.keras.preprocessing.image.array_to_img(process_image_mask(image, pred_mask[0]))
            pred_mask_img.save(unetPP_folder + f'outputs/{index}_predict.png')

            index += 1

# Function to save the images as a plot
def save_sample(display_list, index):
    plt.figure(figsize=(18, 18))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')

    plt.savefig(unetPP_folder + f'outputs/{index}.png')
    plt.show()

## Plot Testing mask & predict mask

In [ ]:
True_Counts = []
Pred_Counts = []

save_predictions(dataset['test'], True_Counts, Pred_Counts)

### Confusion Matrix & Accuracy

In [ ]:
print(True_Counts)
print(Pred_Counts)

num_result = pd.DataFrame({'True_Number': True_Counts, 'Predict_Number': Pred_Counts})
num_result.to_csv(unetPP_folder + 'Number_Result.csv', index= False , header = True)

In [ ]:
def acc_calculation(predict, label):
    total = len(Pred_Counts)
    correct = 0
    for i in range(total):
        if (predict[i] == label[i]):
            correct += 1


    return round(100 * correct / total, 2)

In [ ]:
acc = acc_calculation(Pred_Counts, True_Counts)
print(acc)

num_accuracy = pd.DataFrame({'Number_Accuracy': [acc]})
num_accuracy.to_csv(unetPP_folder + 'Number_Accuracy.csv', index= False , header = True)

In [ ]:
## Generate Confusion Matrix
con = confusion_matrix(True_Counts, Pred_Counts)
print(con)
cnn_df = pd.DataFrame(con)
cnn_df.to_csv(unetPP_folder + 'Number_Conf.csv', index= False , header = False)

print(" ")

nor_con = confusion_matrix(True_Counts, Pred_Counts,normalize='true')
print(nor_con)


nor_con_df = pd.DataFrame(nor_con)
nor_con_df.to_csv(unetPP_folder + 'Number_Conf_Nor.csv', index= False , header = False)

In [ ]:
## [Optional] Load confusion matrix data for plotting
unetPP_folder = 'UNetPP_result/'

con = np.genfromtxt(unetPP_folder + 'Number_Conf.csv', delimiter=',')
print(con)

nor_con = np.genfromtxt(unetPP_folder + 'Number_Conf_Nor.csv', delimiter=',')
print(nor_con)

In [ ]:
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 300


nor_con = np.around(nor_con, 2)
nor_disp = ConfusionMatrixDisplay(confusion_matrix=con)
nor_disp.plot(cmap ='gist_yarg', colorbar=False)
plt.xlabel('Predicted Number', fontsize=12)
plt.ylabel('True Number', fontsize=12)


plt.savefig(unetPP_folder + 'Number_Conf.png', dpi=300, transparent=True, bbox_inches='tight')




In [ ]:
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 300


nor_con = np.around(nor_con, 2)
nor_disp = ConfusionMatrixDisplay(confusion_matrix=nor_con)
nor_disp.plot(cmap ='gist_yarg', colorbar=False)
plt.xlabel('Predicted Number', fontsize=12)
plt.ylabel('True Number', fontsize=12)

plt.savefig(unetPP_folder + 'Number_Conf_Nor.png', dpi=300, transparent=True, bbox_inches='tight')


## Single bacteria analysis

In [ ]:
true_single = np.array(True_Counts)
true_single[true_single != 1] = 0

pred_single = np.array(Pred_Counts)
pred_single[pred_single != 1] = 0


single_result = pd.DataFrame({'True_Single': true_single, 'Pred_Single': pred_single})
single_result.to_csv(unetPP_folder + 'Single_Result.csv', index= False , header = True)



acc = acc_calculation(pred_single, true_single)
print(acc)

single_accuracy = pd.DataFrame({'Single_Accuracy': [acc]})
single_accuracy.to_csv(unetPP_folder + 'Single_Accuracy.csv', index= False , header = True)

In [ ]:
## Generate confusion matrix data
con = confusion_matrix(true_single, pred_single)
print(con)
con_df = pd.DataFrame(con)
con_df.to_csv(unetPP_folder + 'Single_Conf.csv', index= False , header = False)


print( "\n\n\n")

nor_con = confusion_matrix(true_single, pred_single,normalize='true')
print(nor_con)
nor_con_df = pd.DataFrame(nor_con)
nor_con_df.to_csv(unetPP_folder + 'Single_Conf_Nor.csv', index= False , header = False)


In [ ]:
## Optional: Load confusion matrix data
unetPP_folder = 'UNetPP_result/'

con = np.genfromtxt(unetPP_folder + 'Single_Conf.csv', delimiter=',')
print(con)

nor_con = np.genfromtxt(unetPP_folder + 'Single_Conf_Nor.csv', delimiter=',')
print(nor_con)

In [ ]:
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 300

singleBac_labels = np.array(['Non-Single', 'Single-Bacterium'])

disp = ConfusionMatrixDisplay(confusion_matrix=con, display_labels = singleBac_labels)
disp.plot(cmap ='gist_yarg', colorbar=False)
plt.xticks(fontsize = 10)
plt.yticks(rotation=90, ha='right', fontsize = 10, rotation_mode='default', va="center")
plt.xlabel('Prediction', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.savefig(unetPP_folder + 'Single_Conf.png', dpi=300, transparent=True, bbox_inches='tight')


In [ ]:
plt.rcParams['font.size'] = 16
plt.rcParams['figure.dpi'] = 300

singleBac_labels = np.array(['Non-Single', 'Single-Bacterium'])

nor_con = np.around(nor_con, 2)
nor_disp = ConfusionMatrixDisplay(confusion_matrix=nor_con, display_labels = singleBac_labels)
nor_disp.plot(cmap ='gist_yarg', colorbar=False)
plt.xticks(fontsize = 10)
plt.yticks(rotation=90, ha='right', fontsize = 10, rotation_mode='default', va="center")
plt.xlabel('Prediction', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.savefig(unetPP_folder + 'Single_Conf_Nor.png', dpi=300, transparent=True, bbox_inches='tight')

# Model: U-Net (CNN-based)

## Set U-Net result folder

In [ ]:
unet_folder  = 'UNet_result/'

## Real U-net

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


def double_conv_block(x, n_filters):
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   return x

def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = layers.MaxPool2D(2)(f)
   p = layers.Dropout(0.5)(p)
   return f, p


def upsample_block(x, conv_features, n_filters):
   # upsample
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   # concatenate
   x = layers.concatenate([x, conv_features])
   # dropout
   x = layers.Dropout(0.5)(x)
   # Conv2D twice with ReLU activation
   x = double_conv_block(x, n_filters)
   return x


def build_unet_model():
    # inputs
    inputs = layers.Input(shape=(128,128,3))
    # encoder: contracting path - downsample
    # 1 - downsample
    f1, p1 = downsample_block(inputs, 64)
    # 2 - downsample
    f2, p2 = downsample_block(p1, 128)
    # 3 - downsample
    f3, p3 = downsample_block(p2, 256)
    # 4 - downsample
    f4, p4 = downsample_block(p3, 512)
    # 5 - bottleneck
    bottleneck = double_conv_block(p4, 1024)
    # decoder: expanding path - upsample
    # 6 - upsample
    u6 = upsample_block(bottleneck, f4, 512)
    # 7 - upsample
    u7 = upsample_block(u6, f3, 256)
    # 8 - upsample
    u8 = upsample_block(u7, f2, 128)
    # 9 - upsample
    u9 = upsample_block(u8, f1, 64)
    # outputs
    outputs = layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u9)
    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")
    return unet_model









In [ ]:
model = build_unet_model()
model.summary()

## Training


### Loss Function

In [ ]:
m_iou = tf.keras.metrics.BinaryIoU(
    target_class_ids=[1], threshold=0.5, name=None, dtype=None
)

opt = keras.optimizers.Adam(learning_rate=1e-4, clipvalue=0.5)

model.compile(loss=BinaryCrossentropy(), optimizer=opt,  metrics=[m_iou])


## (Optional) Alternate: Load existed model

In [ ]:
model = tf.keras.models.load_model(unet_folder + 'best_model_0510.keras')
model.summary()

## Check Model

In [ ]:
# Function to create a mask out of network prediction
def create_mask(pred_mask: tf.Tensor) -> tf.Tensor:
    # Round to closest
    pred_mask = tf.math.round(pred_mask)
    pred_mask = tf.expand_dims(pred_mask, axis=-1)
    return pred_mask

# Function to show predictions
def show_predictions(dataset=None, num=1):
    if dataset:
        # Predict and show image from input dataset
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display_sample([image[0], mask, create_mask(pred_mask)])
    else:
        # Predict and show the sample image
        inference = model.predict(sample_image)
        display_sample([sample_image[0], sample_mask[0],
                        inference[0]])

for image, mask in dataset['train'].take(1):
    sample_image, sample_mask = image, mask

show_predictions()

## Train Model

In [ ]:
# Callbacks and Logs
class DisplayCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks = [
    DisplayCallback(),
    callbacks.TensorBoard(logdir, histogram_freq = -1),
    callbacks.EarlyStopping(patience = 10, verbose = 1),
    callbacks.ModelCheckpoint(unet_folder +'best_model_0511.keras', verbose = 1, save_best_only = True)

]

## Set Variables
EPOCHS = 300
STEPS_PER_EPOCH = TRAINSET_SIZE // BATCH_SIZE
VALIDATION_STEPS = VALIDSET_SIZE // BATCH_SIZE
print(TRAINSET_SIZE)
print(VALIDSET_SIZE)
print(BATCH_SIZE)
print(STEPS_PER_EPOCH)
print(VALIDATION_STEPS)

要在每次 epoch 後存取這些數據並在訓練結束後進行比較，您可以使用 Keras 的回調函數。以下是步驟來達到這個目的：

創建一個自定義的回調函數來存儲每次 epoch 的數據：
首先，您需要創建一個自定義的回調函數。


In [ ]:
class CustomHistory(Callback):
    def __init__(self):
        self.losses = []
        self.mean_io_u_1_values = []
        self.dice_values = []
        self.val_losses = []
        self.val_mean_io_u_1_values = []
        self.val_dice_values = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs.get('loss'))
        self.mean_io_u_1_values.append(logs.get('binary_io_u'))


        self.val_losses.append(logs.get('val_loss'))
        self.val_mean_io_u_1_values.append(logs.get('val_binary_io_u'))



custom_history = CustomHistory()




In [ ]:
checkpoint = ModelCheckpoint(unet_folder +'best_model_0511.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [custom_history, checkpoint]


initial_time = time()

# Model Train
model.fit(dataset['train'], epochs=EPOCHS,
          steps_per_epoch=STEPS_PER_EPOCH,
          validation_data=dataset['val'],
          validation_steps=VALIDATION_STEPS,
          callbacks=callbacks_list)


print('Finished U-net Training')
print('Training time', time() - initial_time)

model.save(unet_folder + 'epoch300_model/Epoch250_UNET_model.keras')

## validation set  show

In [ ]:
#### Repeat for safety and reabiility #####################################################

# Function to show predictions
def show_predictions(dataset=None, num=1):
    if dataset:
        # Predict and show image from input dataset
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display_sample([image[0], true_mask, create_mask(pred_mask)])
    else:
        # Predict and show the sample image
        inference = model.predict(sample_image)
        display_sample([sample_image[0], sample_mask[0],
                        inference[0]])

#########################################################

for image, mask in dataset['val'].take(1):
    sample_image, sample_mask = image, mask

show_predictions()

## Training loss

In [ ]:
training_loss = custom_history.losses
epochs = list(range(1, len(training_loss) + 1))

df = pd.DataFrame({"Epochs": epochs, "train_loss" : training_loss})
df.to_csv(unet_folder +'training_loss.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

training_loss = custom_history.losses
print(training_loss)

epochs = list(range(1, len(training_loss) + 1))

plt.figure(figsize=(10, 6))
plt.plot(epochs,training_loss, '-', label='training_loss')
plt.title('training_loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('training_loss')
plt.legend()
plt.tight_layout()
plt.show()

## Validate Loss

In [ ]:
val_losses = custom_history.val_losses

epochs = list(range(1, len(val_losses) + 1))

# save log file
df = pd.DataFrame({"Epochs": epochs, "validate_loss" : val_losses})
df.to_csv(unet_folder +'validate_loss.csv', index=False)


plt.figure(figsize=(10, 6))
plt.plot(epochs,val_losses, '-', label='val_losses')
plt.title('val_losses over epochs')
plt.xlabel('Epochs')
plt.ylabel('val_losses')
plt.legend()
plt.tight_layout()
plt.show()


## Mean_ioU

In [ ]:
mean_io_u_1_values = custom_history.mean_io_u_1_values
print(mean_io_u_1_values)

epochs = list(range(1, len(mean_io_u_1_values) + 1))

df = pd.DataFrame({"Epochs": epochs, "train_bIOU" : mean_io_u_1_values})
df.to_csv(unet_folder + 'training_bIOU.csv', index=False)

plt.figure(figsize=(10, 6))
plt.plot(epochs, mean_io_u_1_values, '-', label='mean_io_u_values')
plt.title('mean_io_u over epochs')
plt.xlabel('Epochs')
plt.ylabel('mean_io_u')
plt.legend()
plt.tight_layout()
plt.show()

## Val mean_io_u_1


In [ ]:
val_mean_io_u_1_values = custom_history.val_mean_io_u_1_values
print(val_mean_io_u_1_values)

epochs = list(range(1, len(val_mean_io_u_1_values) + 1))


df = pd.DataFrame({"Epochs": epochs, "val_bIOU" : val_mean_io_u_1_values})
df.to_csv(unet_folder + 'validate_bIOU.csv', index=False)


plt.figure(figsize=(10, 6))
plt.plot(epochs, val_mean_io_u_1_values, '-', label='val_mean_io_u_values')
plt.title('val_mean_io_u_values over epochs')
plt.xlabel('Epochs')
plt.ylabel('val_mean_io_u_values')
plt.legend()
plt.tight_layout()
plt.show()




## 4 in 1 plot

In [ ]:
unet_folder  ='UNet_result/'

# plt.rcParams['font.size'] = 8
# plt.rcParams['figure.dpi'] = 300

df1 = pd.read_csv(unet_folder +'training_loss.csv', header=0)
df2 = pd.read_csv(unet_folder +'validate_loss.csv', header=0)
df3 = pd.read_csv(unet_folder + 'training_bIOU.csv', header=0)
df4 = pd.read_csv(unet_folder + 'validate_bIOU.csv', header=0)



fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('Epochs', fontsize=12)
ax1.set_ylabel('Loss', color=color, fontsize=12)
ax1.plot(df1["Epochs"], df1["train_loss"], '-', label='Training Loss', color=color)
ax1.plot(df2["Epochs"], df2["validate_loss"], '-', alpha = 0.5, label='Validation Loss', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_box_aspect(0.75)
ax1.set_ylim([-0.01, 0.13])


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('IoU', color=color, fontsize=12)  # we already handled the x-label with ax1
ax2.plot(df3["Epochs"], df3["train_bIOU"], '-', label='Training IoU', color=color)
ax2.plot(df4["Epochs"], df4["val_bIOU"], '-', alpha = 0.5, label='Validation IoU', color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim([-0.1, 0.9])



lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=9)


plt.show()

fig.savefig(unet_folder + 'Unet_4to1.png')


# Testing (Test Dataset)


## (Optional) Load pretrained model

In [ ]:
unet_folder  ='UNet_result/'
model = tf.keras.models.load_model(unet_folder + 'epoch300_model/Epoch250_UNET_model.keras')

model.summary()

## Calculate Time for Test-set prediction

In [ ]:
initial_time = time()

for image, mask in dataset['test']:
    pred_mask = model.predict(image)

print('Finished U-net Testing')
print('Testing time', time() - initial_time)

## Predict mask analysis

### Function to calculate mask over image

In [ ]:
# Function to calculate mask over image
def weighted_img(img, initial_img, α=1., β=0.5, γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

# Function to process an individual image and it's mask
def process_image_mask(image, mask):
    # Round to closest
    mask = tf.math.round(mask)

    # Convert to mask image
    zero_image = np.zeros_like(mask)
    mask = np.dstack((mask, zero_image, zero_image))
    mask = np.asarray(mask, np.float32)

    # Convert to image image
    image = np.asarray(image, np.float32)

    # Get the final image
    final_image = weighted_img(mask, image)

    return final_image

In [ ]:
def bacteria_count(mask):
    mask_img = tf.keras.preprocessing.image.array_to_img(mask)
    img2 = cv2.cvtColor(np.asarray(mask_img), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    bacNum = len(cnts)
    return bacNum

In [ ]:
# Function to save predictions
def save_predictions(dataset, True_Counts, Pred_Counts):
    # Predict and save image the from input dataset
    index = 0
    for batch_image, batch_mask in dataset:
        for image, mask in zip(batch_image, batch_mask):
            print(f"Processing image : {index}")
            pred_mask = model.predict(tf.expand_dims(image, axis = 0))
            save_sample([image, process_image_mask(image, mask), process_image_mask(image, pred_mask[0])], index)


            true_bacNum = bacteria_count(mask)
            True_Counts.append(true_bacNum)
            print("True Bac-Num:    ", true_bacNum)
            print(" ")

            pred_mask_round = tf.math.round(pred_mask[0])
            pred_bacNum = bacteria_count(pred_mask_round)
            Pred_Counts.append(pred_bacNum)
            print("Predict Bac-Num: ", pred_bacNum)
            print(" ")

            img = tf.keras.preprocessing.image.array_to_img(image)
            img.save(unet_folder + f'outputs/{index}_well.png')
            true_mask_img = tf.keras.preprocessing.image.array_to_img(process_image_mask(image, mask))
            true_mask_img.save(unet_folder + f'outputs/{index}_true.png')
            pred_mask_img = tf.keras.preprocessing.image.array_to_img(process_image_mask(image, pred_mask[0]))
            pred_mask_img.save(unet_folder + f'outputs/{index}_predict.png')

            index += 1

# Function to save the images as a plot
def save_sample(display_list, index):
    plt.figure(figsize=(18, 18))

    # title = ['Input Image', 'Predicted Mask']
    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')

    plt.savefig(unet_folder + f'outputs/{index}.png')
    plt.show()

In [ ]:
# os.mkdir("outputs")

True_Counts = []
Pred_Counts = []


save_predictions(dataset['test'], True_Counts, Pred_Counts)

### Confusion Matrix

In [ ]:
print(True_Counts)
print(Pred_Counts)

num_result = pd.DataFrame({'True_Number': True_Counts, 'Predict_Number': Pred_Counts})
num_result.to_csv(unet_folder + 'Number_Result.csv', index= False , header = True)

In [ ]:
def acc_calculation(predict, label):
    total = len(Pred_Counts)
    correct = 0
    for i in range(total):
        if (predict[i] == label[i]):
            correct += 1

    return round(100 * correct / total, 2)

In [ ]:
acc = acc_calculation(Pred_Counts, True_Counts)
print(acc)

num_accuracy = pd.DataFrame({'Number_Accuracy': [acc]})
num_accuracy.to_csv(unet_folder + 'Number_Accuracy.csv', index= False , header = True)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

con = confusion_matrix(True_Counts, Pred_Counts)
print(con)
cnn_df = pd.DataFrame(con)
cnn_df.to_csv(unet_folder + 'Number_Conf.csv', index= False , header = False)


print(" ")

nor_con = confusion_matrix(True_Counts, Pred_Counts,normalize='true')
print(nor_con)


nor_con_df = pd.DataFrame(nor_con)
nor_con_df.to_csv(unet_folder + 'Number_Conf_Nor.csv', index= False , header = False)

### [Optional] Load Num_conf csv

In [ ]:
unet_folder  ='UNet_result/'

con = np.genfromtxt(unet_folder + 'Number_Conf.csv', delimiter=',')
print(con)

nor_con = np.genfromtxt(unet_folder + 'Number_Conf_Nor.csv', delimiter=',')
print(nor_con)

In [ ]:
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 300

BacNum_labels = np.array(['0', '1', '2', '3', '4', '5', '6', '7'])

nor_con = np.around(nor_con, 2)
nor_disp = ConfusionMatrixDisplay(confusion_matrix=con, display_labels = BacNum_labels)
nor_disp.plot(cmap ='gist_yarg', colorbar=False)
plt.xlabel('Predicted Number', fontsize=12)
plt.ylabel('True Number', fontsize=12)

plt.savefig(unet_folder + 'Number_Conf.png', dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 300

BacNum_labels = np.array(['0', '1', '2', '3', '4', '5', '6', '7'])

nor_con = np.around(nor_con, 2)
nor_disp = ConfusionMatrixDisplay(confusion_matrix=nor_con, display_labels = BacNum_labels)
nor_disp.plot(cmap ='gist_yarg', colorbar=False)
plt.xlabel('Predicted Number', fontsize=12)
plt.ylabel('True Number', fontsize=12)
plt.savefig(unet_folder + 'Number_Conf_Nor.png', dpi=300, transparent=True, bbox_inches='tight')


In [ ]:
print(True_Counts)
single_count = np.array(True_Counts)
single_count[single_count != 1] = 0
print(single_count)



In [ ]:
true_single = np.array(True_Counts)
true_single[true_single != 1] = 0

pred_single = np.array(Pred_Counts)
pred_single[pred_single != 1] = 0


single_result = pd.DataFrame({'True_Single': true_single, 'Pred_Single': pred_single})
single_result.to_csv(unet_folder + 'Single_Result.csv', index= False , header = True)



acc = acc_calculation(pred_single, true_single)
print(acc)

single_accuracy = pd.DataFrame({'Single_Accuracy': [acc]})
single_accuracy.to_csv(unet_folder + 'Single_Accuracy.csv', index= False , header = True)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

con = confusion_matrix(true_single, pred_single)
print(con)
con_df = pd.DataFrame(con)
con_df.to_csv(unet_folder + 'Single_Conf.csv', index= False , header = False)


print( "\n\n\n")

nor_con = confusion_matrix(true_single, pred_single,normalize='true')
print(nor_con)
nor_con_df = pd.DataFrame(nor_con)
nor_con_df.to_csv(unet_folder + 'Single_Conf_Nor.csv', index= False , header = False)


## [Optional] Load Single_Bac Conf_matrix Csv data

In [ ]:
unet_folder  ='UNet_result/'

con = np.genfromtxt(unet_folder + 'Single_Conf.csv', delimiter=',')
print(con)

nor_con = np.genfromtxt(unet_folder + 'Single_Conf_Nor.csv', delimiter=',')
print(nor_con)

## Plot Single_bac Confusion matrix

In [ ]:
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 300


con = np.around(con, 2)
disp = ConfusionMatrixDisplay(confusion_matrix=con)
disp.plot(cmap ='gist_yarg', colorbar=False)
plt.xlabel('Prediction', fontsize=16)
plt.ylabel('True', fontsize=16)
plt.savefig(unet_folder + 'Single_Conf.png', dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
plt.rcParams['font.size'] = 16
plt.rcParams['figure.dpi'] = 300

singleBac_labels = np.array(['Non-Single', 'Single-Bacterium'])

nor_con = np.around(nor_con, 2)
nor_disp = ConfusionMatrixDisplay(confusion_matrix=nor_con, display_labels = singleBac_labels)
nor_disp.plot(cmap ='gist_yarg', colorbar=False)
plt.xticks(fontsize = 10)
plt.yticks(rotation=90, ha='right', fontsize = 10, rotation_mode='default', va="center")

plt.xlabel('Prediction', fontsize=12)
plt.ylabel('True', fontsize=12)


plt.savefig(unet_folder + 'Single_Conf_Nor.png', dpi=300, transparent=True, bbox_inches='tight')